In [40]:
from juliacall import Main as jl
import numpy as np
import cupy as cp
from cupyx.scipy.sparse import csr_matrix
# Load Clarabel in Julia
jl.seval('using Clarabel, LinearAlgebra, SparseArrays')
jl.seval('using CUDA, CUDA.CUSPARSE')

# Example: Solve a simple second-order cone programming (SOCP) problem

In [41]:
jl.seval('''
    P = CuSparseMatrixCSR(sparse([2.0 1.0 0.0;
                1.0 2.0 0.0;
                0.0 0.0 2.0]))
    q = CuVector([0, -1., -1])
    A = CuSparseMatrixCSR(SparseMatrixCSC([1. 0 0; -1 0 0; 0 -1 0; 0 0 -1]))
    b = CuVector([1, 0., 0., 0.])

    # 0-cone dimension 1, one second-order-cone of dimension 3
    cones = Dict("f" => 1, "q"=> [3])

    settings = Clarabel.Settings(direct_solve_method = :cudss)
                                    
    solver   = Clarabel.Solver(P,q,A,b,cones, settings)
    Clarabel.solve!(solver)
    
    # Extract solution
    x = solver.solution
''')



-------------------------------------------------------------
           Clarabel.jl v0.10.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 3
  constraints   = 4
  nnz(P)        = 5
  nnz(A)        = 4
  cones (total) = 2
    : Zero        = 1,  numel = 1
    : SecondOrder = 1,  numel = 3

settings:
  linear algebra: direct / cudss, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-12, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost       gap     

>>> Clarabel - Results
Status: SOLVED
Iterations: 6
Objective: 0.7500
Solve time: 32.7ms


# Reoptimization

In [42]:
# Update b vector
bpy = cp.array([2.0, 1.0, 1.0, 1.0], dtype=cp.float64)
bjl = jl.Clarabel.cupy_to_cuvector(jl.Float64, int(bpy.data.ptr), bpy.size)

# "_b" is the replacement of "!" in julia function
jl.Clarabel.update_b_b(jl.solver,bjl)          #Clarabel.update_b!()

# Update P matrix
# Define a new CSR sparse matrix on GPU
Ppy = csr_matrix(cp.array([
    [3.0, 0.5, 0.0],
    [0.5, 2.0, 0.0],
    [0.0, 0.0, 1.0]
], dtype=cp.float64))

# Extract the pointers (as integers)
data_ptr    = int(Ppy.data.data.ptr)
indices_ptr = int(Ppy.indices.data.ptr)
indptr_ptr  = int(Ppy.indptr.data.ptr)

# Get matrix shape and non-zero count
n_rows, n_cols = Ppy.shape
nnz = Ppy.nnz

jl.Pjl = jl.Clarabel.cupy_to_cucsrmat(jl.Float64, data_ptr, indices_ptr, indptr_ptr, n_rows, n_cols, nnz)

jl.Clarabel.update_P_b(jl.solver, jl.Pjl)          #Clarabel.update_P!()

#Solve the new problem without creating memory
jl.Clarabel.solve_b(jl.solver)                  #Clarabel.solve!()

-------------------------------------------------------------
           Clarabel.jl v0.10.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 3
  constraints   = 4
  nnz(P)        = 5
  nnz(A)        = 4
  cones (total) = 2
    : Zero        = 1,  numel = 1
    : SecondOrder = 1,  numel = 3

settings:
  linear algebra: direct / cudss, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-12, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost       gap     

>>> Clarabel - Results
Status: SOLVED
Iterations: 7
Objective: 5.500
Solve time: 33.3ms


In [43]:
# Retrieve the solution from Julia to Python
solution = np.array(jl.solver.solution.x)
print("Solution:", solution)

Solution: [ 2.00000000e+00 -1.52972138e-10  9.99999999e-01]
